# Inlämningsuppgift: Del 2 - Adversarial input attack

Emil Karlström \
emkl19@student.bth.se

Samuel Jonsson \
sajs19@student.bth.se

# Instruktioner

Ändra notebookens platshållare så som era namn- och mailuppgifter samt instruktioner i de olika avsnitten. 

Notebookens underrubriker på nivå 3 (###) är bara ett enkelt exempel på hur er implementation kan delas upp. Ni bör skapa egna rubriker anpassade efter er implementation men lämna alla nivå 1 (#) och 2-rubriker (##) som de är. En viktig del inom data-science är att kunna presentera sina metoder och resultat på ett tydligt sätt. **En ostrukturerad och otydlig notebook kan påverka betygsättningen.**

Notera att t.ex. hyperparameter-tuning och annan optimisering för prestanda inte är nödvändigt i denna uppgift. Så länge modellen fungerar märkvärt bättre än ett "coin flip" så räcker det att ni väljer era hyperparametrar manuellt.

# Uppgiften

Uppgiften går ut på att göra så att bilden på koalan nedan klassificeras som en traktor istället. Givetvis ska bilden på koalan se helt oförändrad ut för det mänskliga ögat då attacken är genomförd. Nedan följer Pythonkod som läser in de båda bilderna som uppgiften bygger på samt läser in en ResNet50-modell som klassiciferar bilderna. Saknar ni några Python-paket, t.ex. TensorFlow så installera dem med en pakethanterarare, t.ex. pip

## Beskrivning av adversarial input attacker

Börja med att förklara vad denna typ av attacker är och hur de fungerar i rapportens sektion 2.1.

## Nedan följer kod för att läsa in bilderna och klassificera dem med ResNet50 modellen

In [ ]:
from matplotlib import pyplot as plt
import numpy as np

import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing import image

from art.utils import to_categorical

tf.compat.v1.disable_eager_execution()

print(f"TensorFlow: {tf.__version__}")

In [ ]:
# Loads the pretrained ResNet50 model
model = ResNet50(weights='imagenet')

In [ ]:
# Load the two images (koala & tractor)
init_image = image.load_img("images/koala.jpeg", target_size=(224, 224))
init_image = image.img_to_array(init_image)

target_image = image.load_img("images/tractor.jpeg", target_size=(224, 224))
target_image = image.img_to_array(target_image)

class_lookup = {105: "koala", 866: "tractor"}

In [ ]:
init_class_id = np.argmax(model.predict(np.array([init_image])))
print(f"Init image class label: {init_class_id} ({class_lookup[init_class_id]})")
plt.imshow(init_image.astype(np.uint))
plt.show()

target_class_id = np.argmax(model.predict(np.array([target_image])))
print(f"Target image class label: {target_class_id} ({class_lookup[target_class_id]})")
plt.imshow(target_image.astype(np.uint))
plt.show()

## Implementation av er attack

Beskriv i er rapport vad just er valda attack kallas och i detalj hur den fungerar (sektion 2.2 i rapporten). Beskriv även kort vilka bibliotek ni använder för att implementera den.

## Er kod för attacken

I cellerna nedan lägger ni in er egen kod för att genomföra attacken.

### Bibliotek

In [ ]:
# Samla era import statements här e.g.
# import matplotlib.pyplot as plt
# import numpy as np
# import pandas as pd
# ...
from art.estimators.classification import KerasClassifier
from art.attacks.evasion import ProjectedGradientDescentNumpy

### Egna funktioner

In [ ]:
# Samla era egna funktioner relaterade till attacken här
# Funktionerna ska kort beskrivas med docstrings och kommentarer så att man förstår syftet och implementationen
# Blir det många bör ni bryta ut dessa i en separat fil

### Förberedelser

In [ ]:
# E.g. inläsning av modell och dataset

keras_estimator = KerasClassifier(model, 
                            clip_values=(0,255))

pogboi = ProjectedGradientDescentNumpy(
    estimator=keras_estimator,
    targeted=True,
    eps=20,
    eps_step=0.5,
    max_iter=100,
    random_eps=True,
    batch_size=1024
)

### Attack

Beskriv attackprocessen kortfattat här

In [ ]:
# Attackutförande

target_id = 866 # Tractor ??

attack_image = init_image.reshape((1, 224, 224, 3))

generated_attack_image = pogboi.generate(
    x = attack_image, 
    y = np.array(target_id).reshape((1,)),
    mask = np.array(1.).repeat(224*224*3).reshape((1,224,224,3))
    )
generated_attack_image = generated_attack_image

### Attackresultat

Summera och förklara era attackresultat i flytande text här

In [ ]:
# Presentera attackens resultat m.h.a. kod här

generated_preds = model.predict(generated_attack_image)
generated_label = np.argmax(generated_preds)
print(f"Generated image class label: {generated_label} ({class_lookup[generated_label]}) ({(100*generated_preds[0][generated_label]):.2f}%)")
plt.imshow(generated_attack_image.reshape((224, 224, 3)).astype(np.uint))
plt.title(f"Generated image class label: {generated_label} ({class_lookup[generated_label]}) ({(100*generated_preds[0][generated_label]):.2f}%)")
plt.show()

plt.imshow(init_image.astype(int))
plt.title("Original image")
plt.show()


## Skyddsåtgärder

Beskriv (i rapportens sektion 2.3) vilka säkerhetsåtgärd(er) ni valt som lämpliga för att skydda mot er attack. Motivera varför ni valt just de(n) metod(erna).

## Implementation av skyddsåtgärder (frivilligt endast för A eller B)

Nedan fyller ni i er kod som implementerar skyddsmekanismen mot er attackmetod. Skriv även en summering av vad skyddsresultatet blev när ni implementerade metoden (beskriv detta i rapportens del 2.4).

### Förberedelser

In [ ]:
# E.g. skapande av ny modell, etc.
from art.defences.preprocessor import JpegCompression
jpeg_compressor = JpegCompression(clip_values=(0,255), quality=60)


### Skydd

Beskriv säkerhetsåtgärden kortfattat här

In [ ]:
# Implementation av skyddsåtgärd
# https://adversarial-robustness-toolbox.readthedocs.io/en/latest/modules/defences/preprocessor.html#jpeg-compression

# Tanken är att vi använder oss av JPEG kompression för att sprida ut 
# de attackerade pixlarna över flera pixlar för att minska dess effekt
# på modellen :)
generated_jpeg = np.array(jpeg_compressor(generated_attack_image)[0])

### Skyddsresultat

Bevisa och förklara era skyddsresultat i flytande text här

In [ ]:
# Presentera skyddåtgärdens resultat m.h.a. kod här

# Predict the label
preds_jpeg = model.predict(generated_jpeg)
label_jpeg = np.argmax(preds_jpeg)

# Plot the image
plt.imshow(generated_jpeg.reshape((224, 224, 3)).astype(np.uint))
plt.title(f"JPEG Compressed label: {label_jpeg} ({class_lookup[label_jpeg]}) ({(100*preds_jpeg[0][label_jpeg]):.2f}%)")
plt.show()

plt.imshow(generated_attack_image.reshape((224, 224, 3)).astype(np.uint))
plt.title(f"Generated image class label: {generated_label} ({class_lookup[generated_label]}) ({(100*generated_preds[0][generated_label]):.2f}%)")
plt.show()

plt.imshow(init_image.astype(int))
plt.title("Original image")
plt.show()

# Och det fungerar faktiskt ganska bra :^)

# Referenser


1. Föreläsningar i kursen
2. https://adversarial-robustness-toolbox.readthedocs.io/en/latest/modules/attacks/evasion.html#projected-gradient-descent-pgd-numpy
3. https://adversarial-robustness-toolbox.readthedocs.io/en/latest/modules/defences/preprocessor.html#jpeg-compression